<a href="https://colab.research.google.com/github/SaumyaRaj188/Lunar-Lander-AI/blob/main/Lunar_Lander_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-6).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [86]:
import gymnasium as gym
import numpy as np

In [87]:
!pip install swig
!pip install gym[box2d]

In [88]:
env = gym.make("LunarLander-v2")

In [89]:
from stable_baselines3 import PPO

In [90]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [91]:
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=0)

In [92]:
model.learn(total_timesteps=50_000)

In [93]:
import os
save_dir = "/content/tmp/"
os.makedirs(save_dir, exist_ok=True)

In [94]:
model.save(save_dir + "ppo_lunarlander")

In [95]:
obs = model.env.observation_space.sample()

In [96]:
model.predict(obs, deterministic=True)

(array(1), None)

In [97]:
del model

In [98]:
loaded_model = PPO.load(f"{save_dir}/ppo_lunarlander")

print("loaded", loaded_model.predict(obs, deterministic=True))

loaded (array(1), None)


In [99]:
mean_reward, std_reward = evaluate_policy(loaded_model, env, n_eval_episodes=10, warn=False)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward: -192.78 +/- 53.59


In [100]:
from stable_baselines3.common.vec_env import VecVideoRecorder

In [101]:
def record_video(env_id, model, video_length=500, prefix="", video_folder="/content/videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

In [102]:
record_video("LunarLander-v2", loaded_model, video_length=2000, prefix="lunar_lander")

Saving video to /content/videos/lunar_lander-step-0-to-step-2000.mp4
Moviepy - Building video /content/videos/lunar_lander-step-0-to-step-2000.mp4.
Moviepy - Writing video /content/videos/lunar_lander-step-0-to-step-2000.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/lunar_lander-step-0-to-step-2000.mp4
